<a href="https://colab.research.google.com/github/Molten-Ice/Deep-Learning/blob/dev/GPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("Hi")

Hi


In [4]:
with open("foundation.txt", 'r', encoding='utf-8') as f:
  text = f.read()

In [5]:
print(f"Length of dataset in characters: {len(text)}")

Length of dataset in characters: 1240703


In [6]:
print(text[:1000])

FOUNDATION 
ISAAC ASIMOV 


Contents 

Introduction 

Part I The Psvchohistorians 

Part II The Encyclopedists 

Part III The Mayors 

Part IV The Traders 

Part V The Merchant Princes 

PART I 

THE PSYCHOHISTORIANS 

i. 

HARI SELDON-... bom In the 1 1,988th year of the Galactic Era; died 12,069. The dates are 
more commonly given In terms of the current Foundational Era as - 79 to the year 1 F.E. Born 
to middle-class parents on Flelicon, Arcturus sector (where his father, In a legend of doubtful 
authenticity, was a tobacco grower in the hydroponic plants of the planet), he early showed 
amazing ability in mathematics. Anecdotes concerning his ability are innumerable, and some 
are contradictory. At the age of two, he is said to have ... 

... Undoubtedly his greatest contributions were in the field of psychohistory. Seldon found the 
field little more than a set of vague axioms; he left it a profound statistical science.... 

... The best existing authority we have for the details

In [17]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#%'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ\abcdefghijklmnopqrstuvwxyz—‘’”
84


In [26]:
# create a mapping from characters to integers
stoi = {i:ch for ch, i in enumerate(chars)}
itos = {ch:i for ch, i in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #encode, take a string, output a list of integers
decode = lambda l: ''.join([itos[n] for n in l])#decode, take a list of integers, output a string

print(encode("Hello world!"))
print(decode(encode("Hello world!")))

[34, 58, 65, 65, 68, 1, 76, 68, 71, 65, 57, 2]
Hello world!


In [28]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1240703]) torch.int64
tensor([32, 41, 47, 40, 30, 27, 46, 35, 41, 40,  1,  0, 35, 45, 27, 27, 29,  1,
        27, 45, 35, 39, 41, 48,  1,  0,  0,  0, 29, 68, 67, 73, 58, 67, 73, 72,
         1,  0,  0, 35, 67, 73, 71, 68, 57, 74, 56, 73, 62, 68, 67,  1,  0,  0,
        42, 54, 71, 73,  1, 35,  1, 46, 61, 58,  1, 42, 72, 75, 56, 61, 68, 61,
        62, 72, 73, 68, 71, 62, 54, 67, 72,  1,  0,  0, 42, 54, 71, 73,  1, 35,
        35,  1, 46, 61, 58,  1, 31, 67, 56, 78, 56, 65, 68, 69, 58, 57, 62, 72,
        73, 72,  1,  0,  0, 42, 54, 71, 73,  1, 35, 35, 35,  1, 46, 61, 58,  1,
        39, 54, 78, 68, 71, 72,  1,  0,  0, 42, 54, 71, 73,  1, 35, 48,  1, 46,
        61, 58,  1, 46, 71, 54, 57, 58, 71, 72,  1,  0,  0, 42, 54, 71, 73,  1,
        48,  1, 46, 61, 58,  1, 39, 58, 71, 56, 61, 54, 67, 73,  1, 42, 71, 62,
        67, 56, 58, 72,  1,  0,  0, 42, 27, 44, 46,  1, 35,  1,  0,  0, 46, 34,
        31,  1, 42, 45, 51, 29, 34, 41, 34, 35, 45, 46, 41, 44, 35, 27, 40, 45,
      

In [38]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
print(decode(x.item() for x in train_data[:100]))

FOUNDATION 
ISAAC ASIMOV 


Contents 

Introduction 

Part I The Psvchohistorians 

Part II The Ency


In [39]:
block_size = 8
train_data[:block_size+1]

tensor([32, 41, 47, 40, 30, 27, 46, 35, 41])

In [40]:
for t in range(block_size):
  context = train_data[:t+1]
  target = train_data[t+1]
  print(f"when the input is {context} the target is {target}")

when the input is tensor([32]) the target is 41
when the input is tensor([32, 41]) the target is 47
when the input is tensor([32, 41, 47]) the target is 40
when the input is tensor([32, 41, 47, 40]) the target is 30
when the input is tensor([32, 41, 47, 40, 30]) the target is 27
when the input is tensor([32, 41, 47, 40, 30, 27]) the target is 46
when the input is tensor([32, 41, 47, 40, 30, 27, 46]) the target is 35
when the input is tensor([32, 41, 47, 40, 30, 27, 46, 35]) the target is 41


In [52]:
torch.manual_seed(3)
batch_size = 4
block_size = 8

def get_batch(split="train"):
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data)-block_size, (batch_size, ))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch()
xb.shape, yb.shape
xb, yb

(tensor([[54, 67, 57,  1, 73, 74, 71, 67],
         [68, 69, 65, 58,  1, 56, 68, 67],
         [67, 68,  1, 72, 62, 60, 67, 72],
         [ 1, 42, 71, 62, 73, 56, 61, 58]]),
 tensor([[67, 57,  1, 73, 74, 71, 67, 58],
         [69, 65, 58,  1, 56, 68, 67, 73],
         [68,  1, 72, 62, 60, 67, 72,  1],
         [42, 71, 62, 73, 56, 61, 58, 71]]))

In [53]:
print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

----
when input is [54] the target: 67
when input is [54, 67] the target: 57
when input is [54, 67, 57] the target: 1
when input is [54, 67, 57, 1] the target: 73
when input is [54, 67, 57, 1, 73] the target: 74
when input is [54, 67, 57, 1, 73, 74] the target: 71
when input is [54, 67, 57, 1, 73, 74, 71] the target: 67
when input is [54, 67, 57, 1, 73, 74, 71, 67] the target: 58
when input is [68] the target: 69
when input is [68, 69] the target: 65
when input is [68, 69, 65] the target: 58
when input is [68, 69, 65, 58] the target: 1
when input is [68, 69, 65, 58, 1] the target: 56
when input is [68, 69, 65, 58, 1, 56] the target: 68
when input is [68, 69, 65, 58, 1, 56, 68] the target: 67
when input is [68, 69, 65, 58, 1, 56, 68, 67] the target: 73
when input is [67] the target: 68
when input is [67, 68] the target: 1
when input is [67, 68, 1] the target: 72
when input is [67, 68, 1, 72] the target: 62
when input is [67, 68, 1, 72, 62] the target: 60
when input is [67, 68, 1, 72, 62

In [54]:
print(xb) # input to the transformer

tensor([[54, 67, 57,  1, 73, 74, 71, 67],
        [68, 69, 65, 58,  1, 56, 68, 67],
        [67, 68,  1, 72, 62, 60, 67, 72],
        [ 1, 42, 71, 62, 73, 56, 61, 58]])


In [62]:
try:
    import einops
except:
    print(f"einops not installed as required, installing...")
    !pip3 install einops
    import einops

einops not installed as required, installing...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 3.4 MB/s eta 0:00:00


In [103]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from einops import rearrange, reduce, repeat
torch.manual_seed(3)

class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    #Each token directly reads off the logit for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):

    logits = self.token_embedding_table(idx) # (B, T, C) batch, time, channel
    #[4, 8, 84])
    if targets == None:
      loss = None
    else:
      logits = rearrange(logits, 'B T C -> (B T) C')
      targets = rearrange(targets, 'B T -> (B T)')
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    #idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      #get the predicitions
      logits, loss = self(idx)

      #Focus on last time step
      logits = logits[:, -1, :] # becomes (B, C)
      #apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      #sample from the distribution
      
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx


m = BigramLanguageModel()
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype = torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 84])
tensor(4.9284, grad_fn=<NllLossBackward0>)

av3/—cT'A68XN/LE%L‘‘kkXeW)5d;ZE1s”-,zM8WJMh-GIVZ 
.””TY4f9nRG4kGd—kCy;p%9*/9yB
DaFwiR7sH64e7QLWgV,,H


In [104]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [113]:
batch_size=32
for steps in range(10000):

  #sample a batch of data
  xb, yb = get_batch('train')

  #evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.5057151317596436


In [114]:
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


here. stund s o h bure is waratonat I is 
" FOcron ple 

my 
undengequre feresthardwecte sky. 
nkerin tesqu we lld kesthit looftt, ffow tr." 
Pr s lle weroullof reve wiots Q3: th,"D, blo cadysctheted d aickedld det" iven?"MMUTHe wiave yovowisps bby trsir 
"Warea solouthatifiminolyo sstordacer 

Fask


## The mathematical trick in self-attention

In [128]:
#toy example
B, T, C = 4, 8, 2
x = torch.rand(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [129]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [130]:
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev=x[b,:t+1] #(t, C)
    xbow[b,t]=torch.mean(xprev, 0)
xbow.shape

torch.Size([4, 8, 2])

In [131]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C)
xbow2.shape, xbow2

(torch.Size([4, 8, 2]), tensor([[[0.2566, 0.7936],
          [0.5987, 0.4634],
          [0.7106, 0.5068],
          [0.7503, 0.5220],
          [0.7485, 0.5035],
          [0.7713, 0.5152],
          [0.6992, 0.5313],
          [0.6455, 0.5200]],
 
         [[0.2969, 0.8317],
          [0.2011, 0.5506],
          [0.2537, 0.4335],
          [0.3271, 0.3267],
          [0.4520, 0.2764],
          [0.5243, 0.3275],
          [0.4976, 0.3963],
          [0.5077, 0.4598]],
 
         [[0.5547, 0.3423],
          [0.5945, 0.3534],
          [0.6331, 0.5510],
          [0.6721, 0.4836],
          [0.6954, 0.5048],
          [0.7052, 0.4532],
          [0.6052, 0.4323],
          [0.5441, 0.4920]],
 
         [[0.6440, 0.7071],
          [0.6511, 0.5992],
          [0.7312, 0.4477],
          [0.6812, 0.3755],
          [0.6758, 0.3659],
          [0.6721, 0.3709],
          [0.7067, 0.3470],
          [0.6436, 0.3289]]]))

In [132]:
torch.allclose(xbow, xbow2)

True

In [127]:
xbow[0],xbow2[0]

(tensor([[0.2463, 0.3751],
         [0.3538, 0.4432],
         [0.2456, 0.5110],
         [0.2506, 0.6251],
         [0.3025, 0.5521],
         [0.2629, 0.4743],
         [0.3585, 0.5094],
         [0.4385, 0.4650]]), tensor([[0.2463, 0.3751],
         [0.3538, 0.4432],
         [0.2456, 0.5110],
         [0.2506, 0.6251],
         [0.3025, 0.5521],
         [0.2629, 0.4743],
         [0.3585, 0.5094],
         [0.4385, 0.4650]]))

In [133]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [138]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
print(wei)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


True

In [153]:
# version 4: self-attention (my attempt)
# afternote, need to linearly project to get q,k,v at the start

torch.manual_seed(3)
B, T, C = 4, 8, 32 #batch, time, channels
x = torch.randn(B, T, C)

tril = torch.tril(torch.ones(T,T))
#Query: What am I looking for
#Key: What do I contain
Q, K, V = torch.randn(B, T, C), torch.randn(B, T, C), torch.randn(B, T, C)
wei = Q @ rearrange(K, 'B T C -> B C T')
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ V
wei.shape, wei[0][:3]

(torch.Size([4, 8, 8]),
 tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [3.1798e-03, 9.9682e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [2.6990e-01, 1.2181e-03, 7.2888e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [9.9784e-01, 2.9594e-10, 2.1637e-03, 3.2028e-09, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [9.9892e-01, 4.9244e-09, 1.0709e-03, 7.5832e-06, 2.9531e-08, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [6.2417e-05, 8.8304e-01, 1.1058e-05, 3.7488e-02, 7.9403e-02, 4.6288e-09,
          0.0000e+00, 0.0000e+00],
         [1.9431e-03, 9.9065e-01, 7.3533e-03, 3.4129e-08, 2.2347e-08, 6.9649e-08,
          5.4451e-05, 0.0000e+00],
         [2.9305e-08, 4.7222e-06, 1.8114e-12, 9.7270e-01, 2.7292e-02, 1.1133e-08,
          1.2229e-06, 6.8476e-07]]))

In [157]:
out.shape, out[0][:3]

(torch.Size([4, 8, 32]),
 tensor([[-5.3579e-01,  2.2411e+00, -1.0500e+00,  2.6850e-01,  8.9653e-01,
          -2.2838e-01,  1.7261e+00, -9.0125e-01, -9.8739e-02, -9.4091e-01,
          -1.7678e+00,  1.7666e-01,  4.8473e-01, -2.0531e-01,  2.2039e+00,
          -6.0018e-01,  1.3860e+00, -6.2178e-01, -1.4932e+00,  1.2155e+00,
           4.9699e-02,  7.8623e-01, -6.7045e-01,  1.1508e+00,  5.9142e-01,
          -2.7558e+00,  1.0268e+00, -1.0699e+00, -4.2805e-01,  2.3318e+00,
          -3.3652e-01,  4.0867e-01],
         [ 1.1780e+00,  1.4106e+00, -2.2665e-01,  1.3992e+00, -3.9619e-01,
          -1.4221e+00, -5.7917e-01,  7.4531e-01, -3.9628e-03, -1.7195e+00,
          -4.6205e-01,  6.5826e-01, -8.9226e-01, -2.2795e-01,  2.4093e+00,
           2.2689e+00, -3.5392e-01,  1.2998e+00,  6.3739e-01,  1.3966e+00,
          -7.3228e-01, -1.0222e+00,  1.0640e+00,  2.1083e-01, -1.2269e-02,
           1.4459e+00,  6.3121e-01,  7.8775e-01, -1.5463e-01,  2.0462e-01,
          -4.7826e-01,  2.8160e-01],
 

In [155]:
# version 4: self-attention (Andrej version)

In [163]:
torch.manual_seed(3)
B,T,C=4,8,32
x=torch.randn(B,T,C)

#single self-attention head
head_size=16
key=nn.Linear(C,head_size,bias=False)
query=nn.Linear(C,head_size,bias=False)
value=nn.Linear(C,head_size,bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
v = value(x)

wei = q @ rearrange(k, 'B T C -> B C T')
tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [160]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3835, 0.6165, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1507, 0.2281, 0.6212, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0507, 0.0230, 0.7846, 0.1417, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2544, 0.2024, 0.4529, 0.0681, 0.0223, 0.0000, 0.0000, 0.0000],
        [0.1009, 0.5172, 0.1604, 0.1195, 0.0186, 0.0835, 0.0000, 0.0000],
        [0.1076, 0.0954, 0.0025, 0.0086, 0.7683, 0.0080, 0.0097, 0.0000],
        [0.2585, 0.0457, 0.0329, 0.0113, 0.0118, 0.3126, 0.2737, 0.0535]],
       grad_fn=<SelectBackward0>)